In [15]:
from modeltraining import main
from peft import LoraConfig

rank = 16
peft_config = LoraConfig(r=rank, lora_alpha=rank*2, lora_dropout=0.05, target_modules=['value','query','key', 'dense'])
model_name = 'all-MiniLM-L6-v2'
data_name = 'data.csv'
args_dict = {
    'model_path': f'sentence-transformers/{model_name}',
    'data_path': f'./dataset/{data_name}',
    'device': 'cuda',
    'peft_config': peft_config,
    'batch_size': 16,
    'lr': 5e-5,
    'num_epochs': 5,
    'max_len': 256,
    'eval_every': 999999999,
    'save_model_every': 1000,
    'save_model_path': './models/LoRa',
    'model_save_name': f'LoRa_{model_name}_rank_{rank}'
}

In [16]:
main(args_dict, use_argparse=False)

Arguments:
model_path: sentence-transformers/all-MiniLM-L6-v2
data_path: ./dataset/data.csv
device: cuda
peft_config: LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, base_model_name_or_path=None, revision=None, task_type=None, inference_mode=False, r=16, target_modules=['value', 'query', 'key', 'dense'], lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)
batch_size: 16
lr: 5e-05
num_epochs: 5
max_len: 256
eval_every: 999999999
save_model_every: 1000
save_model_path: ./models/LoRa
model_save_name: LoRa_all-MiniLM-L6-v2_rank_16
triplet_loss: None
shuffle: True
eval_data_path: None
wandb_project_name: None
Loading model...
Loading Peft model From HuggingFace Hub...:  sentence-transformers/all-MiniLM-L6-v2


c:\Users\Edin\anaconda3\envs\amazon\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Edin\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


trainable params: 675840 || all params: 23389056 || trainable%: 2.8895565515769426
Loading data...


Creating triplets:   0%|          | 0/14 [00:00<?, ?group/s]

Creating triplets:   0%|          | 0/12 [00:00<?, ?group/s]

...

Tokenizing data:   0%|          | 0/15 [00:00<?, ?batch/s]

Initializing wandb...


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ammarnasr. Use `wandb login --relogin` to force relogin


Problem at: c:\Users\Edin\OneDrive - University of Edinburgh\Study\deepfund\DeepFunding_project\TripletLoss\modeltraining.py 116 train


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 main(args_dict, use_argparse=False)                                                          │
│   2                                                                                              │
│                                                                                                  │
│ c:\Users\Edin\OneDrive - University of                                                           │
│ Edinburgh\Study\deepfund\DeepFunding_project\TripletLoss\modeltraining.py:68 in main             │
│                                                                                                  │
│    65 │   print('Arguments:')                                                                    │
│    66 │   for key in args_dict.keys():                                                           │
│    67 │   │   print(f'{key}: {args_dict[key]}')                                                  │
│ ❱  68 │   train(**args_dict)                                                                     │
│    69                                                                                            │
│    70 def get_train_eval_data(data_path,train_size=0.9, random_state=42):                        │
│    71 │   #Read CSV and drop nan values                                                          │
│                                                                                                  │
│ c:\Users\Edin\OneDrive - University of                                                           │
│ Edinburgh\Study\deepfund\DeepFunding_project\TripletLoss\modeltraining.py:116 in train           │
│                                                                                                  │
│   113 │   eval_dataset = TripletDataset(val_df, tokenizer=tokenizer, device=device, batch_size   │
│   114 │                                                                                          │
│   115 │   print('Initializing wandb...')                                                         │
│ ❱ 116 │   wandb.init(project=wandb_project_name)                                                 │
│   117 │   wandb.config.update(                                                                   │
│   118 │   │   {                                                                                  │
│   119 │   │   │   'model_path': model_path,                                                      │
│                                                                                                  │
│ c:\Users\Edin\anaconda3\envs\amazon\lib\site-packages\wandb\sdk\wandb_init.py:1173 in init       │
│                                                                                                  │
│   1170 │   except KeyboardInterrupt as e:                                                        │
│   1171 │   │   assert logger                                                                     │
│   1172 │   │   logger.warning("interrupted", exc_info=e)                                         │
│ ❱ 1173 │   │   raise e                                                                           │
│   1174 │   except Exception as e:                                                                │
│   1175 │   │   error_seen = e                                                                    │
│   1176 │   │   traceback.print_exc()                                                             │
│                                                                                                  │
│ c:\Users\Edin\anaconda3\envs\amazon\lib\site-packages\wandb\sdk\wandb_init.py:1150 in init       │
│                                                            